In [3]:
import pandas as pd
import os

In [2]:
directory = ['balance-sheet', 'cash-flow-statement', 'financial-statements', 'financial-ratios']

In [15]:
ticker_to_df = {}
for d in directory:
    # iterate through files in directory
    for b in os.listdir(d):
        for filename in os.listdir(d+ '/' +b):
            # Read csv file
            df = pd.read_csv(d+ '/' + b + '/' + filename)
            ticker = filename.split("-")[0]
            # Add suffix to column names of dataframe

            df.columns = ['Date'] + [str(col) + "-" + b for col in df.columns[1:]]
            # Add dataframe to dictionary
            if ticker not in ticker_to_df:
                ticker_to_df[ticker+"-"+b] = df
            else:
                ticker_to_df[ticker+"-"+b] = pd.merge(ticker_to_df[ticker], df, on='Date', how='outer')
            


In [16]:
# Save df as csv file
for ticker in ticker_to_df:
    ticker_to_df[ticker].to_csv('combined/' + ticker + '.csv', index=False)